In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt
from torch import nn
from torchvision import transforms

## Dowonloading Our going modular

In [2]:
# Try To get torchinfo to get summary
try:
  from torchinfo import summary
except:
  print(f'[INFO] Not Founded , Installing it .............')
  !pip install torchinfo
  from torchinfo import summary

[INFO] Not Founded , Installing it .............


In [3]:
# Try To install Our going modular
try:
  from going_modular.going_modular import data_setup, engine
except:
  print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
  !git clone https://github.com/mrdbourke/pytorch-deep-learning
  !mv pytorch-deep-learning/going_modular .
  !rm -rf pytorch-deep-learning
  from going_modular.going_modular import data_setup, engine

[INFO] Couldn't find going_modular scripts... downloading them from GitHub.
Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 4056, done.
remote: Counting objects: 100% (4056/4056), done.
remote: Compressing objects: 100% (1626/1626), done.
remote: Total 4056 (delta 2387), reused 3946 (delta 2369), pack-reused 0
Receiving objects: 100% (4056/4056), 649.87 MiB | 32.29 MiB/s, done.
Resolving deltas: 100% (2387/2387), done.
Updating files: 100% (248/248), done.


## Dwonload `Pizza, Steak, Sushi` dataset

In [4]:
import os
import zipfile

from pathlib import Path

import requests

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it...
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

    # Download pizza, steak, sushi data
    with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        print("Downloading pizza, steak, sushi data...")
        f.write(request.content)

    # Unzip pizza, steak, sushi data
    with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
        print("Unzipping pizza, steak, sushi data...")
        zip_ref.extractall(image_path)

    # Remove .zip file
    os.remove(data_path / "pizza_steak_sushi.zip")

Did not find data/pizza_steak_sushi directory, creating one...
Unzipping pizza, steak, sushi data...


In [5]:
# Setup dir for data
train_dir = '/content/data/pizza_steak_sushi/train'
test_dir = '/content/data/pizza_steak_sushi/test'

## Turning Our Data into dataloaderwith transformers


1.   Manual transformers
2.   Auto transformers



In [6]:
# Manual transformers
from torchvision import transforms

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

manual_transforms = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    normalize
])

manual_transforms

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)

In [7]:
# Creating DataLoader from manual_transforms
from going_modular.going_modular import data_setup

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                            test_dir=test_dir,
                                                            transform=manual_transforms,
                                                            batch_size=32)

train_dataloader, test_dataloader


(<torch.utils.data.dataloader.DataLoader at 0x7e79e080eef0>,
 <torch.utils.data.dataloader.DataLoader at 0x7e79e080ee00>)

In [8]:
# auto transformers
from torchvision import transforms

# Creating a transorms
weigths = torchvision.models.EfficientNet_B0_Weights.DEFAULT # Best avaliable weigths
weigths

EfficientNet_B0_Weights.IMAGENET1K_V1

In [9]:
# Get auto_transforms from weigths
auto_transforms = weigths.transforms
auto_transforms

functools.partial(<class 'torchvision.transforms._presets.ImageClassification'>, crop_size=224, resize_size=256, interpolation=<InterpolationMode.BICUBIC: 'bicubic'>)

In [10]:
# Creating DataLoader from manual_transforms
from going_modular.going_modular import data_setup

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                            test_dir=test_dir,
                                                            transform=auto_transforms,
                                                            batch_size=32)

train_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x7e79e080d630>,
 <torch.utils.data.dataloader.DataLoader at 0x7e79e080d450>)

## Setting Up pretrained model


In [11]:
device = 'cuda' if torch.cuda.is_available else 'cpu'
device

'cuda'

In [13]:
# Initialize model with the best available weights
weights = torchvision.models.EfficientNet_B1_Weights.DEFAULT # Best avaliable weigths
model = torchvision.models.efficientnet_b1(weights=weights).to(device)
model

Downloading: "https://download.pytorch.org/models/efficientnet_b1-c27df63c.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b1-c27df63c.pth
100%|██████████| 30.1M/30.1M [00:00<00:00, 125MB/s]


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [15]:
model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)

In [20]:
# Getting a summary of our model
from torchinfo import summary

summary(model=model,
        input_size=(1, 3, 224, 224),
        col_names=['input_size', 'output_size', 'num_params', 'trainable'],
        col_width=20,
        row_settings=['var_names'])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 224, 224]     [1, 1000]            --                   True
├─Sequential (features)                                      [1, 3, 224, 224]     [1, 1280, 7, 7]      --                   True
│    └─Conv2dNormActivation (0)                              [1, 3, 224, 224]     [1, 32, 112, 112]    --                   True
│    │    └─Conv2d (0)                                       [1, 3, 224, 224]     [1, 32, 112, 112]    864                  True
│    │    └─BatchNorm2d (1)                                  [1, 32, 112, 112]    [1, 32, 112, 112]    64                   True
│    │    └─SiLU (2)                                         [1, 32, 112, 112]    [1, 32, 112, 112]    --                   --
│    └─Sequential (1)                                        [1, 32, 112, 112]    [1, 16, 112,

## Freezing base layers and make feature_extraction

In [23]:
# Freeze all layers
for param in model.features.parameters():
  param.requires_grad = False

In [25]:
summary(model=model,
        input_size=(1, 3, 224, 224),
        col_names=['trainable'],
        col_width=20,
        row_settings=['var_names'])

Layer (type (var_name))                                      Trainable
EfficientNet (EfficientNet)                                  Partial
├─Sequential (features)                                      False
│    └─Conv2dNormActivation (0)                              False
│    │    └─Conv2d (0)                                       False
│    │    └─BatchNorm2d (1)                                  False
│    │    └─SiLU (2)                                         --
│    └─Sequential (1)                                        False
│    │    └─MBConv (0)                                       False
│    │    └─MBConv (1)                                       False
│    └─Sequential (2)                                        False
│    │    └─MBConv (0)                                       False
│    │    └─MBConv (1)                                       False
│    │    └─MBConv (2)                                       False
│    └─Sequential (3)                                      

In [26]:
model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)

In [27]:
model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=len(class_names), bias=True)).to(device)

In [28]:
summary(model=model,
        input_size=(1, 3, 224, 224),
        col_names=['input_size', 'output_size', 'num_params', 'trainable'],
        col_width=20,
        row_settings=['var_names'])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 224, 224]     [1, 3]               --                   Partial
├─Sequential (features)                                      [1, 3, 224, 224]     [1, 1280, 7, 7]      --                   False
│    └─Conv2dNormActivation (0)                              [1, 3, 224, 224]     [1, 32, 112, 112]    --                   False
│    │    └─Conv2d (0)                                       [1, 3, 224, 224]     [1, 32, 112, 112]    (864)                False
│    │    └─BatchNorm2d (1)                                  [1, 32, 112, 112]    [1, 32, 112, 112]    (64)                 False
│    │    └─SiLU (2)                                         [1, 32, 112, 112]    [1, 32, 112, 112]    --                   --
│    └─Sequential (1)                                        [1, 32, 112, 112]    [1, 1

## Training Our pretrained Model


In [29]:
# loss and optim
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(params=model.parameters(),
                         lr=0.001)

In [39]:
# Creating DataLoader from manual_transforms
from going_modular.going_modular import data_setup

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                            test_dir=test_dir,
                                                            transform=auto_transforms,
                                                            batch_size=32)

train_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x7e79aa35d870>,
 <torch.utils.data.dataloader.DataLoader at 0x7e79aa35c6d0>)

In [42]:
# training model
from timeit import default_timer as timer
from going_modular.going_modular import engine

# Set manual seed
torch.manual_seed(42)
# torch.cuda.manual_seed(42)

# Strat Timer
start_time = timer()


# Setup training and save the results
results = engine.train(model=model,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optim,
                       loss_fn=loss_fn,
                       epochs=5,
                       device=device)

# End Timer
end_time = timer()
print(f'[INFO] Total Train Time : {end_time - start_time :0.3f} s')

  0%|          | 0/5 [00:00<?, ?it/s]

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.10/dist-packages/torchvision/datasets/folder.py", line 231, in __getitem__
    sample = self.transform(sample)
TypeError: ImageClassification.__init__() takes 1 positional argument but 2 positional arguments (and 3 keyword-only arguments) were given
